# Laboratorio 4

### Estimación de Modelos Bid y Choice de Localización

Ayudante: Janus Leonhardt | jaleonhardt@uc.cl 

Profesor: Ricardo Hurtubia | rhurtubia@uc.cl

##### Objetivos del Laboratorio

> 1. Continuar el flujo de los laboratorios anteriores (L1, L2, L3) para estimar un modelo de localización (bid/choice) usando los datos consolidados en celdas y agentes para un periodo de estimación.

> 2. Definir el periodo de estimación y filtrar tanto el GeoDataFrame de celdas (con variables representativas del año base) como los agentes (construcciones realizadas en ese lapso).

> 3. Explicar la formulación logit multinomial para la elección de localización (*choice model*) y el enfoque de subasta (*bid model*)

> 4. Implementar en Python, con `xlogit` (para el modelo choice) y `biogeme` (para el modelo bid), la estimación de dichos modelos y analizar los resultados.

---

### 1. Repaso de Laboratorio 3

En el **Laboratorio 3**, aprendimos a calcular accesibilidades con OSRM mediante un modelo gravitacional, agregando columnas de accesibilidad en nuestras celdas. En este laboratorio, estimaremos un modelo de localización, tanto desde el enfoque **bid** (subasta) como **choice** (elección discreta), asumiendo que tenemos datos de los agentes localizados en un periodo de estimación (2010–2018) y variables de las celdas en ese año base.

---

### 2. Instalación y Configuración del Entorno de Trabajo

#### Requisitos Previos

- Python 3.7 o superior

- Jupyter Notebook

- **Librerías de Python**:

  - pandas  
  - geopandas  
  - shapely  
  - h3  
  - matplotlib  
  - folium  
  - mapclassify  
  - seaborn
  - requests
  - **xlogit**
  - **biogeme**

#### Instalación de Librerías

Recordemos que las librerías deben haber quedado correctamente instaladas en nuestro entorno virtual (`.conda` o `.venv`) creado en el Laboratorio 1. En caso contrario, se debe ejecutar la siguiente línea de código.

`!pip install pandas geopandas shapely h3 matplotlib folium mapclassify seaborn requests xlogit biogeme`

In [1]:
#instalar las librerías faltantes
!pip install xlogit biogeme

### 3. Carga de Datos Previos

En este laboratorio, partiremos desde los archivos generados en los Laboratorios 1 y 3, pero adaptados a la estructura que necesitamos para implementar correctamente los modelos de localización respectivos. Así, los archivos que necesitaremos serán los siguientes.

1. **`agentes.geojson`** (basado en el **Laboratorio 1**): Contiene información de los agentes (líneas de construcción del Servicio de Impuestos Internos), con los siguientes atributos.

   - `año`: el año de construcción.

   - `destino`: el uso de suelo principal de la línea de construcción (habitacional, comercial, etc.).

   - `h3`: la celda *h3* en que se ubica.

   - otras variables catastrales (`superficie_construida`, `calidad`, etc.).

2. **`celdas_estimacion.geojson`** (basado en el **Laboratorio 3**): Contiene información agregada por celda *h3* para el año de inicio del período de estimación (por ejemplo, 2010–2018), incluyendo las características del entorno construido, las accesibilidades gravitacionales del Laboratorio 3 y los datos externos del Laboratorio 2. 

La idea es que estos datos sean **coherentes** con el **año de inicio** del período de estimación. Para el ejemplo, usaremos 2010–2018 como ventana temporal en la que se construyeron/agregaron agentes. Es decir, veremos cuáles construcciones nuevas aparecieron (en `agentes.geojson`) entre 2010 y 2018, y estudiaremos cómo eligieron su localización con base en las características de las celdas en 2010 (contenidas en `celdas_estimacion.geojson`).

In [2]:
import geopandas as gpd

# cargar celdas de estimación y crear id
celdas_estimacion = gpd.read_file("celdas_estimacion.geojson")  
celdas_estimacion['id_celda'] = celdas_estimacion.index

# cargar agentes y filtrar para el periodo de estimación (2010 a 2018)
agentes_estimacion = gpd.read_file("../lab_01/agentes.geojson")
agentes_estimacion = agentes_estimacion[(agentes_estimacion['año'] >= 2010) & (agentes_estimacion['año'] <= 2018)]

# (opcional) quedarnos con las observaciones que están en las celdas de estimación
agentes_estimacion = agentes_estimacion[agentes_estimacion['h3'].isin(celdas_estimacion['h3'])]

# (opcional) quedarnos sólo con uso habitacional, por ejemplo:
agentes_estimacion = agentes_estimacion[agentes_estimacion['destino']=='habitacional']
agentes_estimacion['id_agente'] = agentes_estimacion.reset_index().index

# (opcional) definir la categoría del hogar (1,2,3) según 'calidad'
agentes_estimacion['cat_hogar'] = agentes_estimacion.apply(lambda row: 1 if row['calidad'] <= 2 else (2 if row['calidad'] <= 3 else 3), axis=1)

# ver cuántos agentes habitacionales hay en el periodo de estimación
print(f"Total Agentes Habitacionales: {len(agentes_estimacion)}")

Total Agentes Habitacionales: 12759


---

### 4. Modelo Choice (con *`xlogit`*)

El modelo Choice asume que cada agente $n$ elige una celda $i$ para localizarse, entre un coniunto de celdas. Para ello, definimos la utilidad de la celda $i$ para el agente $n$ de la siguiente manera.

$U_{n,i} = \beta_1 \cdot X_{n,i,1} + \beta_2 \cdot X_{n,i,2} + \dots + \epsilon_{n,i}$

En este caso, $\epsilon_{n,i}$ sigue una distribución Gumbel, y la probabilidad de que el agente *n* escoja la celda $i$ se define a continuación.

$P_{n}(i) \;=\; \frac{\exp(U_{n,i})}{\sum_{k}\exp(U_{n,k})}$

Para su implementación en *xlogit*, se requiere que los datos de entrada estén en *long format* con, al menos, las siguientes columnas.

- ids: identifica al agente $n$.

- alts: identifica la alternativa (es decir, el conjunto de celdas no escogidas).

- chosen: 1 si la celda es la efectivamente elegida, 0 caso contrario.

- variables explicativas: (p. ej., accesibilidad, precio, etc.).

#### Preparación de la Base de Datos *Choice*

En el siguiente ejemplo, consideraremos un conjunto de variables explicativas. Para cada agente, marcaremos su celda real escogida con chosen=1. Luego muestreamos un subconjunto de celdas como “no elegidas”, que se deberá determinar siguiendo un muestreo de alternativas (profundizaremos sobre ello en el Laboratorio 6).

In [3]:
# definir variables explicativas
variables_explicativas_choice = [
    'acc_auto_m2_comercio', 
    'acc_auto_m2_departamento',
    'acc_auto_n_deporte_y_recreacion', 
    'acc_auto_n_educacion_y_cultura',
    'acc_auto_m2_habitacional', 
    'acc_auto_m2_industria',
    'acc_auto_m2_oficina', 
    'acc_auto_n_salud']

# extraer DataFrame con las columnas de celdas que usaremos de explicativas
df_celdas = celdas_estimacion[['h3'] + variables_explicativas_choice].copy()

# crear DataFrame con la celda elegida
df_elegida = agentes_estimacion[['id_agente', 'h3']].assign(chosen=1)
df_elegida = df_elegida.merge(df_celdas, on='h3', how='left')

# crear el conjunto de elección (producto cartesiano entre agentes y celdas)
all_celdas = df_celdas.assign(key=1)
all_agents = agentes_estimacion[['id_agente']].assign(key=1)
bd_choice = all_agents.merge(all_celdas, on='key', how='left').drop('key', axis=1)

# unir con la información real de la celda elegida
bd_choice = bd_choice.merge(df_elegida[['id_agente', 'h3', 'chosen']], on=['id_agente', 'h3'], how='left')
bd_choice['chosen'] = bd_choice['chosen'].fillna(0)

# reemplazar valores nulos en las variables explicativas con 0
bd_choice[variables_explicativas_choice] = bd_choice[variables_explicativas_choice].fillna(0)

# ver DataFrame final
bd_choice.head()

,id_agente,h3,acc_auto_m2_comercio,acc_auto_m2_departamento,acc_auto_n_deporte_y_recreacion,acc_auto_n_educacion_y_cultura,acc_auto_m2_habitacional,acc_auto_m2_industria,acc_auto_m2_oficina,acc_auto_n_salud,chosen
0,0,89ce85a099bffff,8.331040e-24,1.485871e-25,1.414359e-27,1.384718e-26,49.980827,4.093294e-24,1.381523e-24,4.011188e-28,0.0
1,0,89ce85a2917ffff,4.082277e-24,7.280888e-26,6.930475e-28,6.785233e-27,102.000000,2.005747e-24,6.769574e-25,1.965515e-28,0.0
2,0,89ce85a5d0bffff,1.330194e-06,3.843165e-08,3.054418e-12,1.692350e-08,90.063735,3.440712e-05,3.958601e-07,8.408440e-03,0.0
3,0,89ceaa8049bffff,4.560167e+02,1.537733e-01,2.228349e-04,6.923251e-02,1842.976679,9.845618e+02,2.046421e+02,2.869446e-04,0.0
4,0,89ceaa820b7ffff,9.317509e-02,3.141864e-05,2.265586e-04,1.428046e-05,126.118811,2.012358e-01,4.181280e-02,1.445588e-04,0.0


#### Estimación del Modelo *Choice*

A continuación, proporcionaremos todos los argumentos requeridos al método .fit y ajustar así correctamente nuestro modelo *choice* en `xlogit`. Cabe señalar que el modelo Choice estimado en este laboratorio es representativo, considerando que se requiere incorporar un **modelo hedónico de precios a partir de una Regresión Lineal Múltiple** usando `statsmodels` u otras librerías similares.

In [4]:
from xlogit import MultinomialLogit
import pickle

# ajustar el modelo con xlogit
choice_model = MultinomialLogit()
choice_model.fit(
    X=bd_choice[variables_explicativas_choice],
    y=bd_choice['chosen'],
    varnames=variables_explicativas_choice,
    ids=bd_choice['id_agente'],
    alts=bd_choice['h3'])

# guardar el modelo en .pickle
with open("choice.pickle", "wb") as file:
    pickle.dump(choice_model, file)

# mostrar resumen del modelo
choice_model.summary()

Optimization terminated successfully.
    Message: The gradients are close to zero
    Iterations: 12
    Function evaluations: 13
Estimation time= 25.1 seconds
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
acc_auto_m2_comerci    -0.0000716     0.0000022   -33.2729513     7.23e-233 ***
acc_auto_m2_departa    -0.0000429     0.0000063    -6.7858051      1.21e-11 ***
acc_auto_n_deporte_    -0.1521959     0.0101487   -14.9965456      2.08e-50 ***
acc_auto_n_educacio     0.0553482     0.0025667    21.5635505     2.52e-101 ***
acc_auto_m2_habitac     0.0000107     0.0000004    27.3558081     3.66e-160 ***
acc_auto_m2_industr     0.0000081     0.0000013     5.9799191      2.29e-09 ***
acc_auto_m2_oficina    -0.0000807     0.0000079   -10.2434077      1.58e-24 ***
acc_auto_n_salud        0.0935268     0.0109883    

---

### 5. Modelo Bid (con *`biogeme`*)

El modelo Bid asume que la localización es “ganada” por una categoría / uso de suelo. A modo representativo, en este laboratorio consideraremos que sólo los agentes habitacionales "pujan" por una localización, por lo cual definiremos las categorías $c \in \{1,2,3\}$ en función de la calidad de la construcción. Con ello, definimos la disposición a pagar (*willingness to pay*, WTP) de cada categoría  $c$  por una localización  $i$  de la siguiente manera.

$WTP_{c,i} = \beta_{c,0} + \beta_{c,1} \cdot X_{i,1} + \beta_{c,2} \cdot X_{i,2} + \dots$

En este caso,

- $\beta_{c,k}$  son los coeficientes a estimar para la categoría  $c$  y variable $k$.
- $X_{i,k}$  son las variables explicativas de la localización  $i$.

La probabilidad de que la categoría  $c$  gane la subasta por la localización  $i$  se define a continuación.

$P_{i}(c) = \frac{\exp(WTP_{c,i})}{\sum_{k} \exp(WTP_{k,i})}$

Para evitar colinealidad, fijamos el intercepto de una categoría (por ejemplo,  $c = 1$ ) a cero.

#### Preparación de la Base de Datos *Bid*

En el siguiente ejemplo, consideraremos un conjunto de variables explicativas para preparar nuestro modelo Bid para `biogeme`.

In [5]:
import biogeme.database as db

# seleccionar las columnas relevantes para el modelo bid
variables_explicativas_bid = ['acc_auto_m2_comercio', 'acc_auto_n_educacion_y_cultura', 'acc_auto_m2_habitacional', 'acc_auto_m2_industria', 'acc_auto_n_salud']

# preparar la base de datos para biogeme
bd_bid = agentes_estimacion.merge(celdas_estimacion, on='h3', how='left', suffixes=('_ag','_cell'))

# seleccionar solo las columnas necesarias
bd_bid = bd_bid[['id_agente', 'superficie_construida', 'id_celda', 'cat_hogar'] + variables_explicativas_bid]

# definir la base de datos para biogeme
database = db.Database('Bid', bd_bid)
globals().update(database.variables)

#### Estimación del Modelo *Bid*

A continuación, proporcionaremos todos los argumentos requeridos al método .estimate y ajustar así correctamente nuestro modelo *bid* en `biogeme`. Nuevamente, es importante recordar que el modelo Bid estimado en este laboratorio es representativo, considerando que **se requiere estudiar diferentes especificaciones que combinen accesibilidades en diferentes modos de transporte**, de acuerdo a lo revisado en el Laboratorio 3. 

In [6]:
from biogeme import models
from biogeme.expressions import Beta
import biogeme.biogeme as bio

# definir las variables y categorías
xh = ['utilidad']
zi = variables_explicativas_bid
total_vars = xh+zi
categorias = 3

# definir los parámetros a estimar
betas = {}
for c in range(1, categorias + 1):
    for v in total_vars:
        if c == 1:
            fixed = 1
        else:
            fixed = 0
        aux_beta = Beta(f'B_{v}_H{c}', 0, -100, 100, fixed)
        betas.update({f'B_{v}_H{c}': aux_beta})

# definir la disposición a pagar de cada categoría
V = {}
av = {}
for c in range(1, categorias + 1):
    DP_aux = 0
    for v in xh:
        DP_aux += betas[f'B_{v}_H{c}']
    for v in zi:
        DP_aux += betas[f'B_{v}_H{c}']*database.variables[v]
    V.update({c: DP_aux})
    av.update({c: 1})

# definir la log-verosimilitud
logprob = models.loglogit(V, av, cat_hogar)

# (opcional) definir el peso de cada observación
bio.WEIGHT = superficie_construida

# crear el objeto biogeme y estimar el modelo
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'bid'
bid_model = biogeme.estimate()

# guardar el modelo en .pickle
with open("bid.pickle", "wb") as file:
    pickle.dump(bid_model, file)

Con esto completamos el Laboratorio 4 sobre estimación de modelos de localización basados en un enfoque *bid* y *choice* con las librerías `xlogit` y `Biogeme` de Python.

**Fin del Laboratorio 4**

---

En el próximo laboratorio, exploraremos cómo utilizar nuestros estimadores de los modelos de localización para simular nuevas localizaciones futuras.